In [1]:
%pip install -q petals

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import re
import json
import torch
from tqdm import tqdm, trange
from transformers import BloomTokenizerFast
from petals import DistributedBloomForCausalLM
from random import shuffle
import sys

In [5]:
!git clone https://github.com/openai/grade-school-math.git

Cloning into 'grade-school-math'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 36 (delta 10), reused 7 (delta 7), pack-reused 16
Unpacking objects: 100% (36/36), 3.13 MiB | 3.68 MiB/s, done.


In [9]:
PROMPT_FILE_PATH = '/content/drive/MyDrive/socratic_CoT_prompt'
RESULTS_FILE_PATH = '/content/drive/MyDrive/output_self-consistency_method'
ITERATIONS_COUNT = 8
PARAMS = {
    "do_sample": True,
    "temperature": 0.7,
    "top_k": 40,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 100,
    "num_return_sequences": None,
    "stop": ["Q:"]
}
TASK_COUNT = 100

In [7]:
def make_prompt(cot_prompt, cot_answers):
    prompt = ''
    for example, answer in zip(cot_prompt, cot_answers):
        prompt += 'Q: ' + example['question'] + '\nA: ' + example['answer'] + 'The answer is ' + str(answer) + '\n\n'
    with open(PROMPT_FILE_PATH, 'w') as prompt_file:
        prompt_file.write(prompt)
    return prompt

In [8]:
def get_input_and_prompt(test_path, prompt):
    with open(test_path, 'r') as data_file:
        test_problems = data_file.readlines()
    # extract final answer
    answer_list = [float(re.search(r'#### ([0-9-]+)', problem).group(1)) for problem in test_problems]
    # remove annotations of calculator and final answer, convert string to dict
    input_list = [json.loads(re.sub('<<[0-9\(\)\.=+*/-]*>>|#### ([0-9-]+)', '', problem))
                  for problem in test_problems]
    combined = list(zip(answer_list, input_list))
    shuffle(combined)
    answer_list[:], input_list[:] = zip(*combined)
    if prompt is not None:
        cot_prompt = prompt
    else:
        cot_prompt = make_prompt(input_list[:11], answer_list[:11])
        answer_list = answer_list[11:]
        input_list = input_list[11:]
    return input_list, answer_list, cot_prompt

In [ ]:
if __name__ == '__main__':
    results = {
            "Parameters_of_generation": PARAMS,
            "Outputs": []
    }
    if RESULTS_FILE_PATH is not None:
        results = json.load(open(RESULTS_FILE_PATH, 'r'))
    with open(PROMPT_FILE_PATH, 'r') as prompt_file:
        prompt = prompt_file.read()
        start_idx = len(results["Outputs"])
    test_path = './grade-school-math/grade_school_math/data/test.jsonl'
    input_list, answer_list, cot_prompt = get_input_and_prompt(test_path, prompt)
    
    MODEL_NAME = "bigscience/bloomz-petals"
    tokenizer = BloomTokenizerFast.from_pretrained(MODEL_NAME)
    model = DistributedBloomForCausalLM.from_pretrained(MODEL_NAME, request_timeout=300, daemon_startup_timeout=120)
    model = model.cuda()
    for input, answer in tqdm(zip(input_list[start_idx:TASK_COUNT], answer_list[start_idx:TASK_COUNT])):
        task_data = {
            "Question" : input['question'],
            "Answer": str(answer),
            "BLOOM_answers": []
        }
        for i in trange(ITERATIONS_COUNT):
            tokenized_input = tokenizer(cot_prompt + 'Q: ' + input['question'] + '\nA: ', return_tensors="pt")[
            "input_ids"].cuda()
            output = model.generate(tokenized_input, **PARAMS)
            result = {}
            shift = len(cot_prompt) + len(input['question']) + 3
            task_data["BLOOM_answers"].append(tokenizer.decode(output[0])[shift:])
        results["Outputs"].append(task_data)
        with open(RESULTS_FILE_PATH, 'w') as output_file:
            json.dump(results, output_file)